<a href="https://colab.research.google.com/github/RafsanJany-44/Machine-School/blob/main/LIME_ROW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

data = 'https://raw.githubusercontent.com/RafsanJany-44/Thesis_Project/main/All_DATA/without_sn_and_Epoch/EEG_HMC.csv'
dataset = pd.read_csv(data)

from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
dataset["Sleep Stage"]=encoder.fit_transform(dataset["Sleep Stage"])

X = dataset.iloc[:, 1:]
y = dataset.iloc[:, 0]

dic_target = {
    0:"N1"
    1:"N2"
    2:"N3"
    3:"R"
    4:"W"
}

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [3]:
X_train.shape, X_test.shape

((72662, 75), (35789, 75))

StandardScaler() returns a numpy array. For better understandings we convert that in to pandas dataframe. StandardScaler() returns a numpy array. For better understanding we convert that in to pandas dataframe. And in separation for X and y, we remove the <b><i>[.values]</i></b>.

In [4]:
X_train2=X_train
X_test2=X_test

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train =sc.fit_transform(X_train)

X_test =sc.fit_transform(X_test)


X_train = pd.DataFrame(X_train, index=X_train2.index, columns=X_train2.columns)
X_test = pd.DataFrame(X_test, index=X_test2.index, columns=X_test2.columns)


**Tuning**

KNN

In [5]:
from sklearn.neighbors import KNeighborsClassifier
KNN_class = KNeighborsClassifier(n_neighbors=7)
KNN_class.fit(X_train, y_train)
y_pred_KNN=KNN_class.predict(X_test)

Radomforest

In [6]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=125, random_state=0)
rf.fit(X_train, y_train)
y_pred_RF=rf.predict(X_test)

xgboost

In [7]:
import xgboost as xgb
xgb_classifier = xgb.XGBClassifier(max_depth=14)
xgb_classifier.fit(X_train.values, y_train.values)  
y_pred_XG = xgb_classifier.predict(X_test.values)

Accurecy

In [8]:
from sklearn.metrics import accuracy_score
print("Accurecy KNN: ",accuracy_score(y_test, y_pred_KNN))
print("Accurecy RF: ",accuracy_score(y_test, y_pred_RF))
print("Accurecy XG: ",accuracy_score(y_test, y_pred_XG))

Accurecy KNN:  0.7498393361088602
Accurecy RF:  0.7600380005029478
Accurecy XG:  0.780602978568834


#LIME

For further usage and Rearch perpose

In [9]:
!pip install lime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 17.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283839 sha256=61f88959444ffb54e8017afa3b1618cfcda182c2c9b538808949a8a243546fed
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [10]:
X_test.iloc[8]

MeanP_Alpha_F4           -0.030086
MedianF_Alpha_F4          0.711134
MeanF_Alpha_F4           -0.587686
Spectral Edge_Alpha_F4   -0.573012
PeakF_Alpha_F4            0.355503
                            ...   
MeanP_Gamma_O2           -0.094238
MedianF_Gamma_O2         -0.771961
MeanF_Gamma_O2           -0.869609
Spectral Edge_Gamma_O2   -0.731878
PeakF_Gamma_O2           -1.281617
Name: 90165, Length: 75, dtype: float64

In [11]:
ex_specie = np.array(X_test.iloc[10]).reshape(1,-1)

In [12]:
print(X_test.iloc[[10]])

       MeanP_Alpha_F4  MedianF_Alpha_F4  MeanF_Alpha_F4  \
16375       -0.073659         -0.608683       -0.463003   

       Spectral Edge_Alpha_F4  PeakF_Alpha_F4  MeanP_Beta_F4  MedianF_Beta_F4  \
16375                -0.48529        -0.72511      -0.125522        -0.353185   

       MeanF_Beta_F4  Spectral Edge_Beta_F4  PeakF_Beta_F4  ...  \
16375      -0.347385              -0.427643      -0.547305  ...   

       MeanP_Delta_O2  MedianF_Delta_O2  MeanF_Delta_O2  \
16375       -0.139184         -0.148649       -0.345546   

       Spectral Edge_Delta_O2  PeakF_Delta_O2  MeanP_Gamma_O2  \
16375               -0.214028       -0.171524       -0.092981   

       MedianF_Gamma_O2  MeanF_Gamma_O2  Spectral Edge_Gamma_O2  \
16375          0.907416        0.475554               -0.042083   

       PeakF_Gamma_O2  
16375        0.462445  

[1 rows x 75 columns]


In [13]:
print("Prediction for KNN: ",KNN_class.predict(ex_specie)[0])
print("Prediction for RF: ",rf.predict(ex_specie)[0])
print("Prediction for XG: ",xgb_classifier.predict(np.array(X_train.iloc[[10]]))[0])
print("Prediction Accual: ",list(y_test.iloc[[10]])[0])

Prediction for KNN:  2
Prediction for RF:  2
Prediction for XG:  4
Prediction Accual:  2


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [14]:
print("Prediction Accual: ",list(y_test.iloc[[10]])[0])

Prediction Accual:  2


In [15]:
import lime
from lime import lime_tabular
explainer = lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=list(dataset.iloc[:0,1:]),
    class_names=['N1','N2','N3','R','W'],
    mode='classification'
)

In [16]:
exp_KNN = explainer.explain_instance(X_test.iloc[10],
                                 KNN_class.predict_proba,
                                 num_features=6,
                                 top_labels=5)

#exp_KNN.show_in_notebook(show_table=True, show_all=False)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [17]:
exp_KNN.save_to_file("KNN.html")

In [18]:
exp_rf = explainer.explain_instance(X_test.iloc[10],
                                 rf.predict_proba,
                                 num_features=6,
                                 top_labels=5)

#exp_rf.show_in_notebook(show_table=True, show_all=False)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [19]:
exp_rf.save_to_file("RandomForest.html")

In [20]:
explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values,
                                                   feature_names=list(dataset.iloc[:0,1:]),
                                                   class_names=['N1','N2','N3','R','W'])

In [21]:
exp_xg = explainer.explain_instance(X_train.iloc[10, :].values,
                                 xgb_classifier.predict_proba,
                                 num_features=6,
                                 top_labels=5)

#exp_xg.show_in_notebook(show_table=True, show_all=False)

In [22]:
exp_xg.save_to_file("XG_boost.html")



---





---



---



#Reaserch Execution
For only this reasearch

Randomly chossing Rows

In [ ]:
dic_target = {
    0:"N1"
    1:"N2"
    2:"N3"
    3:"R"
    4:"W"
}

In [51]:
from sklearn.utils.extmath import row_norms
from tqdm.notebook import tqdm
import random
all=[]
none=[]
onlyRF=[]
onlyXG=[]
onlyKNN=[]

RFKNN=[]
RFXG=[]
KNNXG=[]


for i in tqdm(range(3000,10000)):
  ex_specie = np.array(X_test.iloc[i]).reshape(1,-1)
  knn=KNN_class.predict(ex_specie)[0]
  ranf=rf.predict(ex_specie)[0]
  xg=xgb_classifier.predict(np.array(X_train.iloc[[i]]))[0]
  y=y_test.iloc[i]
  if y==0:
    if y==knn and y==ranf and y==xg:
      all.append(i)

    elif y==knn and y!=ranf and y!=xg:
      onlyKNN.append(i)

    elif y!=knn and y==ranf and y!=xg:
      onlyRF.append(i)

    elif y!=knn and y!= ranf and y==xg:
      onlyXG.append(i)

    elif y==knn and y==ranf and y!=xg:
      RFKNN.append(i)

    elif y!=knn and y==ranf and y==xg:
      RFXG.append(i)

    elif y==knn and y!=ranf and y==xg:
      KNNXG.append(i)

    else:
      none.append(i)

temp_rows=[all,none,onlyRF,onlyXG,onlyKNN,RFKNN,RFXG,KNNXG]
sec=["ALL","NONE","ONLY_RF","ONLY_XG","ONLY_KNN","RF_KNN","RF_XG","KNN_XG"]
rows=[]

for i in temp_rows:
  rows.append(random.choice(i))

  0%|          | 0/7000 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist

In [ ]:
!pip install lime

In [54]:
len(all)

11

In [56]:
lame(all)

  0%|          | 0/11 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does 

In [55]:
import lime
from lime import lime_tabular
import os
from tqdm.notebook import tqdm
feat_num=5


def lame(rows):

  for row in tqdm(rows):
    ex_specie = np.array(X_test.iloc[row]).reshape(1,-1)

    cls_name=['N1','N2','N3','R','W']


    explainer_1 = lime_tabular.LimeTabularExplainer(
        training_data=np.array(X_train),
        feature_names=list(dataset.iloc[:0,1:]),
        class_names=cls_name,
        mode='classification'
    )


    explainer_2 = lime.lime_tabular.LimeTabularExplainer(
        training_data=X_train.values,
        feature_names=list(dataset.iloc[:0,1:]),
        class_names=cls_name,
    )


    exp_KNN = explainer_1.explain_instance(X_test.iloc[row],
                                    KNN_class.predict_proba,
                                    num_features=feat_num,
                                    top_labels=len(cls_name,))



    exp_rf = explainer_1.explain_instance(X_test.iloc[row],
                                    rf.predict_proba,
                                    num_features=feat_num,
                                    top_labels=len(cls_name,))



    exp_xg = explainer_2.explain_instance(X_train.iloc[row, :].values,
                                    xgb_classifier.predict_proba,
                                    num_features=feat_num,
                                    top_labels=len(cls_name,))

    exp_KNN.save_to_file('/content/Result03/'+"___"+str(row)+"___KNN_.html")
    exp_rf.save_to_file('/content/Result03/'+"___"+str(row)+"___RandomForest_.html")
    exp_xg.save_to_file('/content/Result03/'+"___"+str(row)+"___XG_boost_.html")



In [57]:
!zip -r /content/Result03.zip /content/Result03

from google.colab import files
files.download("/content/Result03.zip")

  adding: content/Result03/ (stored 0%)
  adding: content/Result03/___4908___KNN_.html (deflated 80%)
  adding: content/Result03/___3179___XG_boost_.html (deflated 80%)
  adding: content/Result03/___5975___KNN_.html (deflated 80%)
  adding: content/Result03/___3700___KNN_.html (deflated 80%)
  adding: content/Result03/___5818___XG_boost_.html (deflated 80%)
  adding: content/Result03/___4616___KNN_.html (deflated 80%)
  adding: content/Result03/___5818___RandomForest_.html (deflated 80%)
  adding: content/Result03/___9611___RandomForest_.html (deflated 80%)
  adding: content/Result03/___5294___XG_boost_.html (deflated 80%)
  adding: content/Result03/___5975___RandomForest_.html (deflated 80%)
  adding: content/Result03/___8153___XG_boost_.html (deflated 80%)
  adding: content/Result03/___3179___KNN_.html (deflated 80%)
  adding: content/Result03/___8153___KNN_.html (deflated 80%)
  adding: content/Result03/___8117___KNN_.html (deflated 80%)
  adding: content/Result03/___9760___RandomFo

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(rows)
print(sec)

[73, 58, 84, 83, 43, 3, 67, 48]
['ALL', 'NONE', 'ONLY_RF', 'ONLY_XG', 'ONLY_KNN', 'RF_KNN', 'RF_XG', 'KNN_XG']


In [ ]:
print(sec[rows.index(49)])

RF_XG
